In [1]:
from pymmcore_plus import CMMCorePlus
from fov import FOV
import useq
from useq import MDAEvent
from queue import Queue
import numpy as np
from matplotlib import pyplot as plt
from napari import Viewer
import numpy as np
from queue import Queue
from pymmcore_plus import CMMCorePlus
from useq import MDAEvent
import useq
from useq._channel import Channel
from MDAEngine_DMD import MDAEngine_DMD
from controller import Controller, Analyzer
import pandas as pd
import random
from utils import ImgType, MetadataDict
from stimulation import StimExtraParameters
from dmd import DMD
from hardware import load_config
import os
import tempfile
from datetime import datetime
import pymmcore
import napari
from tracking import TrackerTrackpy
from utils import create_folders

Validation errors in config file(s).
The following fields have been reset to the default value:

schema_version
  value is not a valid tuple (type=type_error.tuple)



In [2]:
#TODO check if dmd calibration works
#TODO create calibration widget for napari?
#TODO adjust napari live view, check how they change brightness in napari-umanager (or directly use napari-umanager?)
#TODO add napari widget to store/display positions
#TODO add to known issues: conda install -c conda-forge llvm-openmp if there are issues with threads on M1
#TODO preview the segmentation and mask creation
#TODO store tracks, masks, and segmentation
#TODO binning for camera


## Load uManager and set default devices

In [3]:
mmc = CMMCorePlus() #Create a new instance of the CMMCorePlus class
#mmc.loadSystemConfiguration('local/local_config.cfg')  #Load the system configuration file (standard uManager .cfg file)
#load default config:
mmc.loadSystemConfiguration()
#load_config(mmc) #Set the default device from the YAML file (specific to this package)

In [4]:
# Create a new log file in the temp directory
log_dir = tempfile.gettempdir()
#log_dir = 'Z:\\lhinder\\data\\rtm_mm_data\\exp_355\\'
log_dir = '/Volumes/imaging.data/lhinder/data/rtm_mm_data/exp_355/'
log_file = os.path.join(log_dir, "log.txt")

# Get the current date and time
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print(f"Logfile: {log_file}")

# Open the log file in append mode and write the date and time
with open(log_file, "w") as file: #w: write (delete old content), a: append
    file.write(f"{current_time}\n")
    file.write(f"User ID: {mmc.getUserId()}")

    file.write(f"PyMMCore version {pymmcore.__version__}\n")
    file.write(f"{mmc.getAPIVersionInfo()}")
    file.write(f"{mmc.getVersionInfo()}\n\n")

    file.write("Loaded devices:\n")
    file.write("\n".join(str(device) for device in mmc.getLoadedDevices()))
    file.write("\n\n")

Logfile: /Volumes/imaging.data/lhinder/data/rtm_mm_data/exp_355/log.txt


In [5]:
#viewer = napari.Viewer()
#image_layer = viewer.add_image(np.ones((1,1)))

In [6]:
path = log_dir
create_folders(path,['stim','raw','labels','stim_mask','tracks','labels_rings','particles'])

Directory /Volumes/imaging.data/lhinder/data/rtm_mm_data/exp_355/stim already exists
Directory /Volumes/imaging.data/lhinder/data/rtm_mm_data/exp_355/raw already exists
Directory /Volumes/imaging.data/lhinder/data/rtm_mm_data/exp_355/labels already exists
Directory /Volumes/imaging.data/lhinder/data/rtm_mm_data/exp_355/stim_mask already exists
Directory /Volumes/imaging.data/lhinder/data/rtm_mm_data/exp_355/tracks already exists
Directory /Volumes/imaging.data/lhinder/data/rtm_mm_data/exp_355/labels_rings already exists
Directory /Volumes/imaging.data/lhinder/data/rtm_mm_data/exp_355/particles already exists


## Create list with FOVs

In [7]:
# get current position
pos = (mmc.getXPosition(),mmc.getYPosition())
pos_list = [pos,pos]

In [8]:
print(mmc.getXPosition())
print(mmc.getYPosition())

-0.0
-0.0


## Create a DF with all planned acquisitions and stimulations

In [9]:
range(2)

range(0, 2)

In [10]:
df_acquire = pd.DataFrame(columns=['fov', 'timestep', 'time','time_experiment', 'treatment', 'acquired','stim', 'channels', 'channel_stim'])

time_between_frames = 5 #time in seconds between frames
stim_timesteps = [2,3]  # list of timesteps
#stim_timesteps= []  # list of timesteps

timesteps = range(5)  # 0-20
treatments = ['stim_top', 'stim_mid',]  # list of treatments
channels = [['miRFP','mCherry']]
channels_exposure = [[200,100]]
channel_stim = ['mCitrine']
channel_stim_exposure = [100]

fovs:list[FOV] = []

for i,pos in enumerate(pos_list):
    print(pos)
    fov = FOV(pos=pos,
              index =i,
              path=path,
              metadata={},
              properties={'stim_property': 'top'},
              )
    fovs.append(fov)

# Loop over the FOVs and randomly assign one of the treatments to it
treatments_shuffled = treatments.copy()
random.shuffle(treatments_shuffled)
for fov in fovs:
    treatment = treatments_shuffled[fov.index % len(treatments_shuffled)]
    for timestep in timesteps:
        new_row = { 'fov_object': fov,
                    'fov':fov.index,
                    'timestep': timestep,
                    'time': timestep*time_between_frames,
                    'treatment': treatment,
                    'acquired': False,
                    'stim': timestep in stim_timesteps,
                    'channels': channels,
                    'channels_exposure':channels_exposure,
                    'channel_stim' : channel_stim,
                    'channel_stim_exposure' : channel_stim_exposure,
                    'fname' : f'{str(fov.index).zfill(3)}_{str(timestep).zfill(5)}',
                    
                    ## Custom parameters for the stimulation
                    'offset_x': 0, #offset for the stimulation mask
                    'offset_y': 10, #offset for the stimulation mask
                    'radius': 10, #radius of the stimulation mask
                    }
        df_acquire = pd.concat([df_acquire, pd.DataFrame(new_row, index=[0])])


df_acquire = df_acquire.sort_values(by=['timestep', 'fov'])

df_acquire

(-0.0, -0.0)
(-0.0, -0.0)


,fov,timestep,time,time_experiment,treatment,acquired,stim,channels,channel_stim,fov_object,channels_exposure,channel_stim_exposure,fname,offset_x,offset_y,radius
0,0,0,0,NaN,stim_top,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x2a103d160>,"[200, 100]",100.0,000_00000,0.0,10.0,10.0
0,1,0,0,NaN,stim_mid,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x2a103e100>,"[200, 100]",100.0,001_00000,0.0,10.0,10.0
0,0,1,5,NaN,stim_top,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x2a103d160>,"[200, 100]",100.0,000_00001,0.0,10.0,10.0
0,1,1,5,NaN,stim_mid,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x2a103e100>,"[200, 100]",100.0,001_00001,0.0,10.0,10.0
0,0,2,10,NaN,stim_top,False,True,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x2a103d160>,"[200, 100]",100.0,000_00002,0.0,10.0,10.0
0,1,2,10,NaN,stim_mid,False,True,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x2a103e100>,"[200, 100]",100.0,001_00002,0.0,10.0,10.0
0,0,3,15,NaN,stim_top,False,True,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x2a103d160>,"[200, 100]",100.0,000_00003,0.0,10.0,10.0
0,1,3,15,NaN,stim_mid,False,True,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x2a103e100>,"[200, 100]",100.0,001_00003,0.0,10.0,10.0
0,0,4,20,NaN,stim_top,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x2a103d160>,"[200, 100]",100.0,000_00004,0.0,10.0,10.0
0,1,4,20,NaN,stim_mid,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x2a103e100>,"[200, 100]",100.0,001_00004,0.0,10.0,10.0


## Take an image at the current location

In [11]:
print(f'Channels to acquire: {channels[0]}')

Channels to acquire: ['miRFP', 'mCherry']


## Run on virtual system 

In [12]:
from add_frame import ImageProcessingPipeline
from segmentation import SegmentatorStardist
from pymmcore_plus.mda import MDAEngine
from stimulation import StimExtraParameters, StimCircle
from controller import Analyzer

segmentator = SegmentatorStardist('2D_versatile_fluo')
stimulator = StimExtraParameters()
stimulator = StimCircle()
tracker = TrackerTrackpy()
pipeline = ImageProcessingPipeline(segmentator,stimulator,tracker)
analyzer = Analyzer(pipeline)


Found model '2D_versatile_fluo' for 'StarDist2D'.


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


### TEST MODE

In [13]:
import napari
import zarr
import glob
from skimage.io import imread
from glob import glob
import dask.array as da
from dask import delayed
import os
import numpy as np
from magicgui import magicgui
import pandas as pd



def tiff_to_lazy_da(path,folder,fov, zfill = 2):
    '''Read in all tiff files form the same FOV in a folder and load them lazily with dask. '''
    file_name_pattern = str(fov).zfill(zfill)+"_*.tiff"
    filenames = sorted(glob(path + os.path.join(str(folder),file_name_pattern)))
    # read the first file to get the shape and dtype
    # ASSUMES THAT ALL FILES SHARE THE SAME SHAPE and TYPE

    sample = imread(filenames[0])
    
    lazy_imread = delayed(imread)  # lazy reader
    lazy_arrays = [lazy_imread(fn) for fn in filenames]
    dask_arrays = [
        da.from_delayed(delayed_reader, shape=sample.shape, dtype=sample.dtype)
        for delayed_reader in lazy_arrays
    ]
    # Stack into one large dask.array
    stack = da.stack(dask_arrays, axis=0)
    stack = np.squeeze(stack)
    return stack

project_path = '/Volumes/imaging.data/lhinder/data/rtm_mm_data/exp_352/'
stack_raw = tiff_to_lazy_da(project_path, "raw", 0,zfill=2)
stack_stim = tiff_to_lazy_da(project_path, "stim", 0,zfill=2)

In [14]:

from controller_simulation import Controller, Analyzer

dmd = DMD(mmc, test_mode = True)
mda_engine_dmd = MDAEngine_DMD(dmd)

mmc.mda.set_engine(mda_engine_dmd)

queue = Queue()

### RUN THIS IF YOU WANT TO TEST THE ACQUISITION WITHOUT ACTUAL HARDWARE
analyzer = Analyzer(pipeline)
controller = Controller(analyzer, mmc, queue,stack_raw=stack_raw,stack_stim=stack_stim)


# Start the acquisition
controller.run(df_acquire)


Simulating acquisition with stack of shape:  (150, 2, 1024, 1024)
Max frame in df:  4


In [32]:

tracks = pd.read_pickle('/Volumes/imaging.data/lhinder/data/rtm_mm_data/exp_355/tracks/000_00004.pkl')

In [36]:
tracks['frame']

0     0
1     0
2     0
3     0
4     0
     ..
70    4
71    4
72    4
73    4
74    4
Name: frame, Length: 75, dtype: int64

### REAL MODE

In [34]:
#mmc = CMMCorePlus()

#mmc.loadSystemConfiguration()

#setup camera
#mmc.initializeDevice("Camera");
#mmc.setCameraDevice("Camera");

# Register the custom engine with the runner
dmd = DMD(mmc, test_mode = False)
mda_engine_dmd = MDAEngine_DMD(dmd)

mmc.mda.set_engine(mda_engine_dmd)

queue = Queue()

controller = Controller(analyzer, mmc, queue)

# Start the acquisition
controller.run(df_acquire)
#controller.run_test()

RuntimeError: No device with label ""

In [ ]:
configs = mmc.getAvailableConfigs('Channel')
for config in configs:
    print(config)


GFP
mCherry
mCitrine
miRFP


## Run on system with DMD

In [ ]:
from add_frame import ImageProcessingPipeline
from segmentation import SegmentatorStardist

segmentator = SegmentatorStardist('2D_versatile_fluo')
stimulator = StimExtraParameters()
pipeline = ImageProcessingPipeline(segmentator,stimulator)
analyzer = Analyzer(pipeline)



mmc = CMMCorePlus()
dmd = DMD(mmc)

mmc.loadSystemConfiguration()
mmc.mda.set_engine(MDAEngine_DMD(mmc,dmd))

# Apply the custom acquisition engine
# Register the custom engine with the runner
core = CMMCorePlus.instance()
#core.mda.set_engine(MDAEngine_DMD(core))

# create the Queue that will hold the MDAEvents
queue = Queue()



controller = Controller(analyzer, mmc , queue, dmd)
# Start the acquisition
controller.run()

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


RuntimeError: No device with label ""

In [10]:
def normalize_mi_ma(x, mi, ma, clip=False, eps=1e-20, dtype=np.float32):
    if dtype is not None:
        x   = x.astype(dtype,copy=False)
        mi  = dtype(mi) if np.isscalar(mi) else mi.astype(dtype,copy=False)
        ma  = dtype(ma) if np.isscalar(ma) else ma.astype(dtype,copy=False)
        eps = dtype(eps)

def normalize(x, pmin=3, pmax=99.8, axis=None, clip=False, eps=1e-20, dtype=np.float32):
    """Percentile-based image normalization."""

    mi = np.percentile(x.flatten(),pmin)
    ma = np.percentile(x.flatten(),pmax)
    return normalize_mi_ma(x, mi, ma, clip=clip, eps=eps, dtype=dtype)

normalize(np.ones((1000,1000)))
print(np.ones((1000,1000)).flatten().shape)

(1000000,)


In [4]:
#import napari_umanager
#from napari_umanager import napari_manager
#napari_manager(viewer, mmc, q)

ModuleNotFoundError: No module named 'napari_umanager'